# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
hvplot.extension("bokeh")
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [6]:
geoapify_key

'cef74a1eae544f0da4064de8a2aa35e0'

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,48.74,89,100,7.81,AU,1731004326
1,1,atafu village,-8.5421,-172.5159,83.71,65,6,11.14,TK,1731004327
2,2,longyearbyen,78.2186,15.6401,33.64,75,75,13.80,SJ,1731004413
3,3,puerto natales,-51.7236,-72.4875,46.85,61,100,8.05,CL,1731004415
4,4,iqaluit,63.7506,-68.5145,19.13,79,75,15.01,CA,1731004213


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "Humidity"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0) & (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 90)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,bethel,41.3712,-73.4140,71.69,46,0,12.66,US,1731004438
28,28,rawson,-43.3002,-65.1023,70.09,21,0,29.93,AR,1731004150
42,42,umm kaddadah,13.6017,26.6876,78.40,21,0,15.26,SD,1731004463
47,47,bandarbeyla,9.4942,50.8122,80.31,75,0,14.18,SO,1731004468
68,68,al hazm,16.1641,44.7769,75.45,25,0,11.99,YE,1731004492


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[[
    "City",
    "Country",
    "Lat",
    "Lng",
    "Humidity"
]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
23,bethel,US,41.3712,-73.4140,46,
28,rawson,AR,-43.3002,-65.1023,21,
42,umm kaddadah,SD,13.6017,26.6876,21,
47,bandarbeyla,SO,9.4942,50.8122,75,
68,al hazm,YE,16.1641,44.7769,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit" : 20,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
rawson - nearest hotel: Hotel Deportivo
umm kaddadah - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
bubaque - nearest hotel: Chez Julio
ad dabbah - nearest hotel: No hotel found
shinas - nearest hotel: No hotel found
keren - nearest hotel: Eritrea Hotel قندق إرتريا
alaghsas - nearest hotel: Hôtel Zagado
belmar - nearest hotel: Mayfair Hotel
marsa alam - nearest hotel: Blue House Hotel
aswan - nearest hotel: Oscar Hotel
forest - nearest hotel: Tru by Hilton Lynchburg
saint-pierre - nearest hotel: Tropic Hotel
faya-largeau - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
dubai - nearest hotel: Holiday Inn Bur Dubai - Embassy District
kidal - nearest hotel: No hotel found
copiapo - nearest hotel: Hotel Diego de Almeida
birao - nearest hotel: No hotel found
freetown

,City,Country,Lat,Lng,Humidity,Hotel Name
23,bethel,US,41.3712,-73.4140,46,Hampton Inn Danbury
28,rawson,AR,-43.3002,-65.1023,21,Hotel Deportivo
42,umm kaddadah,SD,13.6017,26.6876,21,No hotel found
47,bandarbeyla,SO,9.4942,50.8122,75,No hotel found
68,al hazm,YE,16.1641,44.7769,25,No hotel found
77,guerrero negro,MX,27.9769,-114.0611,33,Plaza sal paraiso
79,bubaque,GW,11.2833,-15.8333,68,Chez Julio
90,ad dabbah,SD,18.0500,30.9500,24,No hotel found
109,shinas,OM,24.7433,56.4658,50,No hotel found
127,keren,ER,15.7778,38.4581,67,Eritrea Hotel قندق إرتريا


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "Humidity",
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)